https://haystack.deepset.ai/tutorials/40_building_chat_application_with_function_calling

components: 
InMemoryDocumentStore, SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder, InMemoryEmbeddingRetriever, ChatPromptBuilder, OpenAIChatGenerator, ToolInvoker

OpenAPI API key

In [ ]:
import os
openai_key = False
hf_token = False 
with open("secrets") as file:
    for line in file.readlines():
        key,value = line.strip().split("=")
        if key == 'OPENAI_API_KEY':
            openai_key = True
            os.environ[key]=value
        elif key == 'HF_TOKEN':
            hf_token = True
            os.environ[key]=value
assert openai_key, 'OPENAI_API_KEY not found'

# assert hf_token, 'HF_TOKEN not found'
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
#setup OPENAI_API_KEY
from pprint import pprint

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [4]:
messages = [
    ChatMessage.from_system('Always respond in German'),
    ChatMessage.from_user('Briefly explain about the current state of health care in the US'),
]

chat_gen = OpenAIChatGenerator(model="gpt-4o-mini")
# chat_gen = OpenAIChatGenerator(model="gpt-4o-mini", streaming_callback = callback_fn)
res = chat_gen.run(messages=messages)

# document_store = InMemoryDocumentStore(embedding_similarity_function='cosine')
# text_embeder = SentenceTransformersTextEmbedder()
# retriever = InMemoryEmbeddingRetriever(document_store=document_store)

In [ ]:
print(res['replies'][0]._content[0].text)

In [6]:
facts = [
    "The Earth is the only planet in our solar system not named after a god.",
    "The Amazon rainforest produces more than 20% of the world's oxygen supply.",
    "Antarctica is the driest, windiest, and coldest continent.",
    "There are more than 24 time zones around the world.",
    "The Great Wall of China is the longest man-made structure in the world.",
    "Mount Everest is the highest point on Earth.",
    "The Pacific Ocean is the largest and deepest ocean on Earth.",
    "Russia is the largest country by land area.",
    "The Sahara Desert is the largest hot desert in the world.",
    "The Nile River is the longest river in the world."
]

docs = [Document(content=fact) for fact in facts]
doc_store = InMemoryDocumentStore()

In [ ]:
pipeline = Pipeline()
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

pipeline.add_component(
    instance=SentenceTransformersDocumentEmbedder(model=embedding_model), 
    name="doc_embedder"
)

pipeline.add_component(
    name="doc_writer",
    instance = DocumentWriter(document_store=doc_store) 
)

pipeline.connect('doc_embedder.documents', 'doc_writer.documents')

pipeline.run({'doc_embedder': {"documents": docs}})



# RAG
haystack.components.embedders.SentenceTransformersTextEmbedder
haystack.components.retrievers.in_memory.InMemoryEmbeddingRetriever
haystack.components.builders.ChatPromptBuilder
haystack.dataclasses.ChatMessage
haystack.components.generators.chat.OpenAIChatGenerator

In [8]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

In [9]:
template = [
    ChatMessage.from_system(
    """
    Answer the questions based on the given context.

    Context:
    {% for document in documents %}
        {{ document.content }}
    {% endfor %}
    Question: {{ question }}
    Answer:
    """
    )
]

rag = Pipeline()

rag.add_component('embedder', SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
rag.add_component('retriever', InMemoryEmbeddingRetriever(document_store = doc_store))
rag.add_component('prompt_builder', ChatPromptBuilder(template = template))
rag.add_component('llm', OpenAIChatGenerator(model = 'gpt-4o-mini'))

rag.connect('embedder', 'retriever.query_embedding')
rag.connect('retriever', 'prompt_builder.documents')
rag.connect('prompt_builder.prompt', 'llm.messages')

rag.draw("sample.png")


In [ ]:
def ask(question):
    return rag.run({'embedder': {'text': question}, 'prompt_builder': {'question' : question} })


res = ask("What is some facts about earth")


In [ ]:
print(res)
print(res['llm']['replies'][0].text)

# Using tools in Haystack

In [42]:
# Pipeline as a tool
from haystack.tools import Tool

params = {
    'type': 'object',
    'properties': {
        'question': {
            'type': 'string',
            'description': 'Query used for search. Infer this information from user message'
        }
    },
    'required': ['question']
}

rag_tool = Tool(
    name='rag_pipeline_tool',
    description="Get information about provided facts in the document_store",
    parameters=params,
    function = ask,
)

In [43]:
#2. Function as a tool
from typing import Annotated, Literal
from haystack.tools import create_tool_from_function

WEATHER_INFO = {
    "Berlin": {"weather": "mostly sunny", "temperature": 7, "unit": "celsius"},
    "Paris": {"weather": "mostly cloudy", "temperature": 8, "unit": "celsius"},
    "Rome": {"weather": "sunny", "temperature": 14, "unit": "celsius"},
    "Madrid": {"weather": "sunny", "temperature": 10, "unit": "celsius"},
    "London": {"weather": "cloudy", "temperature": 9, "unit": "celsius"},
}


def get_weather(
        city: Annotated[str, 'the city for which to get weather'] = 'Berlin',
        unit: Annotated[Literal["Celsius", "Fahrenheit"], 'the temperature unit'] = 'Berlin'):
    '''A simple function to get the current weather for a location'''
    if city in WEATHER_INFO:
        return WEATHER_INFO[city]
    else:
        return {"weather": "sunny", "temperature": 21.8, "unit": "fahrenheit"}

weather_tool = create_tool_from_function(get_weather)


# Running OpenAIChatGenerator with tools

In [44]:
from haystack.dataclasses import ChatMessage
from haystack.components.tools import ToolInvoker


In [45]:
user_messages = [
    ChatMessage.from_system(
        "Use the tool that you're provided with. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
    ),
    ChatMessage.from_user('Can you tell me 1 interesting fact')
]

res = chat_gen.run(messages = user_messages, tools = [rag_tool, weather_tool])

In [ ]:
pprint(res['replies'][0])

In [ ]:
tool_invoker = ToolInvoker(tools = [rag_tool, weather_tool])

tool_res_message = tool_invoker.run(messages = res['replies'])['tool_messages']
# ToolCallResult is a json. so you can call the tool in your code
print(tool_res_message)


In [ ]:
print(res['replies'])

final_message = user_messages + res['replies'] + tool_res_message

final_rep = chat_gen.run(messages = final_message, tools = [rag_tool, weather_tool])

In [ ]:
pprint(final_rep)